In [1]:
import torch
import torchvision
from torch import nn
from torchvision import transforms
from MLProject2Dataset import *

In [2]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


# ΑΠΛΟ ΣΝΔ

- Input             3 * 50 * 62 = 9300
- 1st conv          32 * 48 * 60 = 92160
- 1st maxpool       32 * 24 * 30 = 23040
- 2nd conv          64 * 22 * 28 = 39424
- 2nd maxpool       64 * 11 * 14 = 9856
- 3rd conv          64 * 9 * 12 = 6912
- 3rd maxpool       64 * 4 * 6 = 1536


In [3]:
# class SimpleCNN(nn.Module):
#     def __init__(self, output_features=5):
#         super().__init__()
        
#         self.stack = nn.Sequential(
#                 nn.Conv2d(3, 32, 3),
#                 nn.ReLU(),
#                 nn.MaxPool2d(2, 2),
#                 nn.Conv2d(32, 64, 3),
#                 nn.ReLU(),
#                 nn.MaxPool2d(2, 2),
#                 nn.Conv2d(64, 64, 3),
#                 nn.ReLU(),
#                 nn.MaxPool2d(2, 2),
#                 nn.Flatten(), # Check parameter
#                 nn.Linear(1536, output_features)
#         )   

#     def forward(self, x):
#         return self.stack(x)
    
# model = SimpleCNN(1)

In [4]:
from torchsummary import summary
from models import *

### Train

In [5]:
dataset = MLProject2Dataset('./data/dermoscopy_classification', transform=image_transforms(50, 62))

generator = torch.Generator().manual_seed(42)
train, validate, test = torch.utils.data.random_split(dataset, [0.6, 0.1, 0.3], generator)

In [6]:
train[0][1]

2

In [7]:
type(train[0][1])

numpy.int64

In [8]:
train_dataloader = DataLoader(train, batch_size=64, shuffle=True)
validate_dataloader = DataLoader(validate, batch_size=64, shuffle=True)

In [9]:
device='cuda'
model = SimpleCNN().to(device)
loss_fn = nn.CrossEntropyLoss().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.0001)

In [10]:
train_fn(model, train_dataloader, validate_dataloader, optimizer = optimizer, loss=loss_fn, device=device)

Epoch: 0, batch:     9] train loss: 1.941 train acc: 0.164 | val loss: 0.031 val acc: 0.000
Epoch: 0, batch:    19] train loss: 1.938 train acc: 0.201 | val loss: 0.031 val acc: 0.000
Epoch: 0, batch:    29] train loss: 1.936 train acc: 0.228 | val loss: 0.031 val acc: 0.000
Epoch: 0, batch:    39] train loss: 1.933 train acc: 0.264 | val loss: 0.031 val acc: 0.000
Epoch: 0, batch:    49] train loss: 1.933 train acc: 0.288 | val loss: 0.031 val acc: 0.000
Epoch: 0, batch:    59] train loss: 1.930 train acc: 0.312 | val loss: 0.031 val acc: 0.000
Epoch: 0, batch:    69] train loss: 1.926 train acc: 0.348 | val loss: 0.031 val acc: 0.000
Epoch: 0, batch:    79] train loss: 1.924 train acc: 0.380 | val loss: 0.031 val acc: 0.000
Epoch: 0, batch:    89] train loss: 1.922 train acc: 0.408 | val loss: 0.031 val acc: 0.000
Epoch: 1, batch:     9] train loss: 1.919 train acc: 0.644 | val loss: 0.031 val acc: 0.000
Epoch: 1, batch:    19] train loss: 1.918 train acc: 0.646 | val loss: 0.031 val

KeyboardInterrupt: 